In [1]:
import random as rd
import numpy as np

from glob import glob

In [2]:
def euclid_extended(a, b):
    '''
    Realiza o cálculo do mdc
    entre a e b, retornando
    o mdc e x e y tais que
    ax + by = mdc(a, b).
    '''
    inverted = False
    if b > a:
        a, b = b, a
        inverted = True
        
    
    table = np.array([[a, b], [1, 0], [0, 1]])
    iteration = 0
    while table[0, (iteration + 1) % 2] != 0:
        a, b = table[0, iteration % 2], table[0, (iteration + 1) % 2]
        q = a // b
        r = a % b
        table[:, iteration % 2] -= table[:, (iteration + 1) % 2] * q
        iteration += 1
        
    lcd, x, y = table[:, iteration % 2]
    
    if inverted:
        return lcd, y, x
    else:
        return lcd, x, y

def is_prime(n):
    '''
    Recebe um número e verifica
    se ele é primo
    '''
    if n == 2:
        return True
    
    if n < 2 or n % 2 == 0:
        return False
    
    for i in range(3, int(n**(0.5)) + 1, 2):
        if n % i == 0:
            return False
        
    return True

In [3]:
# Buscando os primos de 10.000 até
# 19.999

if 'primos.npy' not in glob('*.npy'):
    primes = []
    a = 10000
    for i in range(5000):
        if is_prime(a + 2 * i + 1):
            primes.append(a + 2 * i + 1)

    primes = np.array(primes)
    with open('primos.npy', 'wb') as f:
        np.save(f, primos)

else:
    primes = np.load('primos.npy')

primes

array([10007, 10009, 10037, ..., 19991, 19993, 19997])

In [4]:
def choose_parameters(primes):
    '''
    Recebe um array de primos e
    retorna dois primos p e q,
    phi(n) e números e e d de
    modo que de = 1 (mod phi(n)),
    onde n = p*q
    '''
    
    rd.shuffle(primes)
    p, q = primes[:2]
    phi_n = (p - 1) * (q - 1)
    
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
    
    return p, q, phi_n, e, d

def str2int(message):
    exp = 1
    number = 0
    message = message[::-1]
    for i in range(len(message)):
        number += ord(message[i]) * exp
        exp *= 256
    return number

def int2str(number):
    message = ''
    while number != 0:
        temp = number % 256
        message += chr(temp)
        number -= temp
        number = number // 256
    message = message[::-1]
    return message

def generate_keys(p, q):
    phi_n = (p - 1) * (q - 1)
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
        
    return (phi_n, e), (phi_n, d)

def encrypt(message, public_key):
    phi_n, e = public_key
    phi_n = int(phi_n)
    m = str2int(message)
    m = (m * e) % phi_n
    encrypted = int2str(m)
    
    return encrypted

def decrypt(encrypted, private_key):
    phi_n, d = private_key
    d = int(d)
    phi_n = int(phi_n)
    m = str2int(encrypted)
    m = (m * d) % phi_n
    message = int2str(m)
    
    return message

In [5]:
original_message = 'Hello World!'
# alguns primos de https://en.wikipedia.org/wiki/Largest_known_prime_number
# p, q = 999999000001, 67280421310721
# p, q = 100000000000000003, 100000000000000013
# p, q = 170141183460469231731687303715884105727, 20988936657440586486151264256610222593863921
p, q = 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127, 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151

public_key, private_key = generate_keys(p, q)
m = encrypt(original_message, public_key)
message = decrypt(m, private_key)
print(m)
print(message)

/ÈÜ0PQf&óºÎz KàENIËPg3§ò7LÖ¹¯¼69Õ~±,ìp¾sEY»}e³òö_^vBÈÞ9¬+¾ØEkÍèïÿ°8Î6½Aôñæ~<Æh==:5rDÝY#jÝ#ò à£²WÜS-é
Hello World!


In [7]:
# phi_n, e = public_key
# phi_n, d = private_key
# print((phi_n * _ + e * d) % phi_n)
# print(p, is_prime(p), q, is_prime(q))